In [ ]:
!pip install -q transformers
!pip install -q hazm
!pip install -q PersianStemmer
!pip install -q clean-text[gpl]
!pip install nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/models/persian_talent/claim

In [ ]:
import pandas as pd
data_train = pd.read_csv('-923066623_-294158543.csv')
data_train
data2 = pd.read_csv('153952002_-1390683709 (1).csv')
data_train = pd.concat([data_train,data2],ignore_index=True)
data_train

In [ ]:

data_test = pd.read_csv('4.test_data_without_label_text.csv')

data_test

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from PersianStemmer import PersianStemmer
from hazm import *
from cleantext import clean

from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

import tensorflow_datasets as tfds
import tensorflow as tf

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [ ]:
def clean_sentence(sentence):
    sentence = arToPersianChar(sentence)
    sentence = arToPersianNumb(sentence)
    sentence = remove_number(sentence)
    # more_normalization_function()
    return sentence


def arToPersianNumb(number):
    dic = {
        '١': '۱',
        '٢': '۲',
        '٣': '۳',
        '٤': '۴',
        '٥': '۵',
        '٦': '۶',
        '٧': '۷',
        '٨': '۸',
        '٩': '۹',
        '٠': '۰',
        '1': '۱',
        '2': '۲',
        '3': '۳',
        '4': '۴',
        '5': '۵',
        '6': '۶',
        '7': '۷',
        '8': '۸',
        '9': '۹',
        '0': '۰',

    }
    return multiple_replace(dic, number)

def remove_number(number):
    dic = {
        '۱': '',
        '۲': '',
        '۳': '',
        '۴': '',
        '۵': '',
        '۶': '',
        '۷': '',
        '۸': '',
        '۹': '',
        '۰': '',

    }
    return multiple_replace(dic, number)

def arToPersianChar(userInput):
    dic = {
        'ك': 'ک',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
        }
    return multiple_replace(dic, userInput)


def multiple_replace(dic, text):
    pattern = "|".join(map(re.escape, dic.keys()))
    return re.sub(pattern, lambda m: dic[m.group()], str(text))

In [ ]:
def clean_all(document):
    clean = []
    for sentence in document:
        sentence = clean_sentence(sentence)
        clean.append(sentence)
    return clean

In [ ]:
pesrian_stopwords = []
with open('../persian') as f:
    persian_stopwords = f.readlines()


persian_stopwords = [i[:-1]for i in persian_stopwords]

In [ ]:
persian_stopwords

In [ ]:
lines = data_train.data.tolist()

lines = clean_all(lines)
normalizer = Normalizer()
lines = list(map(lambda line: normalizer.normalize(line), lines))
ps = PersianStemmer()
lemmatizer = Lemmatizer()


train = []

target = []
for line in lines:
    line = re.sub(r"[_,-؛,،؟?!/*#]",' ',line)
    line = ps.run(line)
    words = nltk.tokenize.word_tokenize(line)
    #words = [re.sub(r"[_,-?!/]",' ',word) for word in words]
    #words = [ps.run(word) for word in words]
    words = [word for word in words  if word not in persian_stopwords]
    #words = [lemmatizer.lemmatize(word) for word in words]
    train.append(words)


In [ ]:
lines = data_test.data.tolist()

lines = clean_all(lines)
normalizer = Normalizer()
lines = list(map(lambda line: normalizer.normalize(line), lines))
ps = PersianStemmer()
lemmatizer = Lemmatizer()


test = []

target = []
for line in lines:
    line = re.sub(r"[_,-؛,،؟?!/*#]",' ',line)
    line = ps.run(line)
    words = nltk.tokenize.word_tokenize(line)
    #words = [re.sub(r"[_,-?!/]",' ',word) for word in words]
    #words = [ps.run(word) for word in words]
    words = [word for word in words  if word not in persian_stopwords]
    #words = [lemmatizer.lemmatize(word) for word in words]
    test.append(words)


In [ ]:
print('sample of data befor cleaning:{}'.format(lines[1]))
print('sample of cleaned data:{}'.format(test[1]))


In [ ]:
all_words = set([word for sentence in train for word in sentence])
num_words = len(set([word for sentence in train for word in sentence]))

labels = data_train.columns[3:].tolist()
target = data_train[labels]
num_tags   = len(set(labels))

In [ ]:
target

In [ ]:
print("Total number of tagged sentences: {}".format(len(train)))
print("Vocabulary size: {}".format(num_words))
print("Total number of tags: {}".format(num_tags))

In [ ]:
d = []
for line in train:
  ds = ''
  for i in range(len(line)):
    ds += line[i] +' '
  d.append(ds)

train = pd.DataFrame(data=d,columns=['topic'])
train[labels] = target

In [ ]:
d = []
for line in test:
  ds = ''
  for i in range(len(line)):
    ds += line[i] +' '
  d.append(ds)

test = pd.DataFrame(data=d,columns=['topic'])


In [ ]:
train.head()

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

bert_features=[]
for text in tqdm(train['topic']):

  input_ids = tokenizer.encode(text, add_special_tokens=True)
  input_ids = torch.tensor([input_ids])

  # Pass the tokenized input through BERT
  outputs = model(input_ids)

  # Extract features from the last layer
  last_layer_features = outputs.last_hidden_state.detach().numpy()
  bert_features.append(last_layer_features[0])


In [ ]:
test_bert_features=[]
for text in tqdm(test['topic']):

  input_ids = tokenizer.encode(text, add_special_tokens=True)
  input_ids = torch.tensor([input_ids])

  # Pass the tokenized input through BERT
  outputs = model(input_ids)

  # Extract features from the last layer
  last_layer_features = outputs.last_hidden_state.detach().numpy()
  test_bert_features.append(last_layer_features[0])


In [ ]:
bert_features[1].shape

In [ ]:
bert_pad_size = [len(i) for i in bert_features]
train_pad = [np.pad(i,((0,max(bert_pad_size)-len(i)),(0,0))) for i in bert_features]
test_pad = [np.pad(i,((0,max(bert_pad_size)-len(i)),(0,0))) for i in test_bert_features]


In [ ]:
max(bert_pad_size)

In [ ]:
train_features = pd.DataFrame()
train_features['bert_features'] = train_pad

In [ ]:
test_features = pd.DataFrame()
test_features['bert_features'] = test_pad

In [ ]:
train_features

In [ ]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz
#!gzip -d cc.fa.300.vec.gz


In [ ]:
from gensim.models import KeyedVectors

model_path = "../cc.fa.300.vec"
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=False)




In [ ]:
train

In [ ]:
word_embedding =[]
for text in train['topic']:
  vector=[]
  words = nltk.tokenize.word_tokenize(text)
  for word in words:
    try:
      vector.append(word2vec_model[word])
    except:
      0
  word_embedding.append(vector)


In [ ]:
test_word_embedding =[]
for text in test['topic']:
  vector=[]
  words = nltk.tokenize.word_tokenize(text)
  for word in words:
    try:
      vector.append(word2vec_model[word])
    except:
      0
  test_word_embedding.append(vector)


In [ ]:
w_pad_size = [len(i) for i in word_embedding]
train_w_pad = [np.pad(i,((0,max(w_pad_size)-len(i)),(0,0))) for i in word_embedding]
test_w_pad = [np.pad(i,((0,max(w_pad_size)-len(i)),(0,0))) for i in test_word_embedding]

In [ ]:
len(train_w_pad[2])

In [ ]:
target.to_numpy()

In [ ]:
train_features['word_embedding'] = train_w_pad
train_features['label'] = list(target.to_numpy())
train_features.to_pickle('train_features.pkl')
train_features

In [ ]:
train_features = pd.read_pickle('train_features.pkl')
train_features

In [ ]:
test_features['word_embedding'] = test_w_pad
test_features

In [ ]:
test_features.to_pickle('test_features.pkl')


## loss weight

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K


In [ ]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight(class_weight ='balanced', classes=[0.,1.],  y=y_true[:, i])
    return weights

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss



In [ ]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import numpy as np

def create_cnn_model(input_shape):

    droprate = 0.2
    input_layer = Input(shape=input_shape)

    # CNN layers
    conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
    batch1 = BatchNormalization()(conv1)
    drop1 = Dropout(droprate)(batch1)

    conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(drop1)
    batch2 = BatchNormalization()(conv2)
    drop2 = Dropout(droprate)(batch2)

    conv3 = Conv1D(filters=128, kernel_size=3, activation='relu')(drop2)
    batch3 = BatchNormalization()(conv3)
    drop3 = Dropout(droprate)(batch3)


    # Flatten layer
    flatten = Flatten()(drop3)

    # Classification layers
    dense = Dense(128, activation='relu')(flatten)
    batch4 = BatchNormalization()(dense)
    drop4 = Dropout(droprate)(batch4)

    dense2 = Dense(128, activation='relu')(drop4)
    dense3 = Dense(64, activation='relu')(dense2)


    # Output layer
    outputs = Dense(11, activation='sigmoid')(dense3)
    model = Model(inputs=input_layer, outputs=outputs)


    model.compile(optimizer = tf.keras.optimizers.Adam(0.0001),
              loss = get_weighted_loss(class_weights),
              metrics = ['accuracy',tf.keras.metrics.AUC()])

    return model









In [ ]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers
import tensorflow as tf
import numpy as np

def create_cnn_model(input_shape):

    droprate = 0.2
    input_layer = Input(shape=input_shape)

    # CNN layers
    conv1 = Conv1D(filters=8, kernel_size=10, activation='relu')(input_layer)
    batch1 = BatchNormalization()(conv1)
    drop1 = Dropout(droprate)(batch1)


    # Flatten layer
    flatten = Flatten()(drop1)

    # Classification layers
    dense = Dense(64, activation='relu',bias_regularizer=regularizers.L2(1e-4),kernel_regularizer=regularizers.l2(0.01))(flatten)
    batch4 = BatchNormalization()(dense)
    drop4 = Dropout(droprate)(batch4)

    dense2 = Dense(64, activation='relu',bias_regularizer=regularizers.L2(1e-4),kernel_regularizer=regularizers.l2(0.01))(drop4)
    dense3 = Dense(64, activation='relu',bias_regularizer=regularizers.L2(1e-4),kernel_regularizer=regularizers.l2(0.01))(dense2)


    # Output layer
    outputs = Dense(11, activation='sigmoid')(dense3)
    model = Model(inputs=input_layer, outputs=outputs)


    model.compile(optimizer = tf.keras.optimizers.Adam(0.0001),
              loss = get_weighted_loss(class_weights),
              metrics = ['accuracy',tf.keras.metrics.AUC()])

    return model









In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional,LSTM, Dense
from tensorflow.keras.optimizers import Adam

def create_cnn_model(input_shape):

    droprate = 0.2
    input_layer = Input(shape=input_shape)

    # CNN layers



    conv3 = Bidirectional(LSTM(128))(input_layer)
    batch3 = BatchNormalization()(conv3)
    drop3 = Dropout(droprate)(batch3)


    # Flatten layer
    flatten = Flatten()(drop3)

    # Classification layers
    dense = Dense(128, activation='relu')(flatten)
    batch4 = BatchNormalization()(dense)
    drop4 = Dropout(droprate)(batch4)

    dense2 = Dense(128, activation='relu')(drop4)
    dense3 = Dense(64, activation='relu')(dense2)


    # Output layer
    outputs = Dense(11, activation='sigmoid')(dense3)
    model = Model(inputs=input_layer, outputs=outputs)


    model.compile(optimizer = tf.keras.optimizers.Adam(0.0001),
              loss = get_weighted_loss(class_weights),
              metrics = ['accuracy',tf.keras.metrics.AUC()])

    return model


In [ ]:
train_features['bert_features'] = list(np.array(train_features['bert_features'].to_list())[:,0,:].reshape(-1,768,1))


In [ ]:
train, valid = train_test_split(train_features, test_size=0.15)

In [ ]:
models = ['bert_features','word_embedding']

from sklearn.utils import shuffle


for index,model_name in enumerate(models):
  X_train = np.array(train[model_name].tolist())
  y_train = np.array(train['label'].tolist())
  X_val = np.array(valid[model_name].tolist())
  y_val = np.array(valid['label'].tolist())
  y_train = y_train.astype(np.float32)
  y_val = y_val.astype(np.float32)

  class_weights = calculating_class_weights(y_train)


  input_shape = X_train.shape[1:]  # Replace with your actual input shape

  model = create_cnn_model(input_shape)


  model.fit(X_train, y_train, batch_size = 64, epochs = 100, validation_data = [X_val, y_val],
                      callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)])
  model.save_weights(model_name+".h5")


In [ ]:
import tensorflow as tf
from keras.layers import Dense, Flatten,BatchNormalization,Input,Activation,Dropout
from keras.models import Sequential,Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM

from tensorflow.keras.models import Model
from tensorflow.keras.applications import mobilenet_v2, mobilenet, resnet50, densenet
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, \
    BatchNormalization, Activation, GlobalAveragePooling2D, DepthwiseConv2D, \
    Dropout, ReLU, Concatenate, Input, add, Conv1D, MaxPooling1D
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN,Reshape
from tensorflow.keras.utils import plot_model
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
from keras.regularizers import l1, l2
from tensorflow.keras.callbacks import CSVLogger
from datetime import datetime
from tensorflow.keras.losses import BinaryCrossentropy
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
from keras.models import load_model

def load_all_models(models_name,input_shape):
  model_list = list()
  for index,i in enumerate(models_name):
    name = i+'.h5'
    model = create_cnn_model(input_shape[index])
    model.load_weights(name)
    model_list.append(model)
    print('loaded %s'%i)
  return model_list


def define_stacked_model(members):
	wd = 0.005
	droprate=0.2
	# update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			layer.trainable = True
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i+1) + '_' + layer.name

	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	hidden_dense = Dense(64,kernel_regularizer=l2(wd), activation='relu')(merge)
	output = Dense(11, activation='sigmoid')(hidden_dense)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.compile(optimizer = tf.keras.optimizers.Adam(0.0001),
                loss = get_weighted_loss(class_weights),
                metrics = ['accuracy',tf.keras.metrics.AUC()])
	return model
input_shapes=[np.array(train[i].tolist()).shape[1:] for i in models]
tuned_models = load_all_models(models,input_shapes)
# define ensemble model
stacked_model = define_stacked_model(tuned_models)

In [ ]:
X_train = [np.array(train[i].tolist())for i in models]
y_train = np.array(train['label'].tolist())

X_val = [np.array(valid[i].tolist()) for i in models]
y_val = np.array(valid['label'].tolist())

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

stacked_model.fit(X_train, y_train, batch_size = 64, epochs = 50, validation_data = [X_val, y_val],
                    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20)], shuffle=True)
stacked_model.save("stacked_model3.h5")


## test model

In [ ]:
test_features['bert_features'] = list(np.array(test_features['bert_features'].to_list())[:,0,:].reshape(-1,768,1))


In [ ]:
X_test = [np.array(test_features[i].tolist()) for i in models]
pred = stacked_model.predict(X_test)


In [ ]:
pred[:30]

In [ ]:
y_pred = [[1 if i>0.5 else 0 for i in n] for n in pred]
y_pred[:30]

In [ ]:
data_test

In [ ]:
np.array(y_pred)[:,1]

In [ ]:
for index,i in enumerate(labels):
  data_test[i] = list(np.array(y_pred)[:,index])
data_test.to_csv('claim_final_test4.csv',index=False)
pd.read_csv('claim_final_test4.csv')